## Imports

필요한 라이브러리 Import 

In [ ]:
import numpy as np
import pandas as pd 

from sklearn.preprocessing import LabelEncoder

import os

import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import seaborn as sns

## Data Loda

- 모든 대출에는 자체 행이 있으며 SK_ID_CURR로 식별
- training data의 target 데이터는 0과 1로 나타남
    - 0 : 대출금 상환
    - 1 : 대출금 상환 X

In [ ]:
print(os.listdir("../input/"))

In [ ]:
# Training data
app_train = pd.read_csv('../input/home-credit-default-risk/application_train.csv')
print('Training data shape: ', app_train.shape)

# Train data 확인
app_train.head()

In [ ]:
# Testing data 확인
app_test = pd.read_csv('../input/home-credit-default-risk/application_test.csv')
print('Testing data shape: ', app_test.shape)
app_test.head()

# EDA

## train의 'target' 데이터 비율 살펴보기

In [ ]:
app_train['TARGET'].value_counts()

In [ ]:
# 히스토그램으로 보기
app_train['TARGET'].astype(int).plot.hist();

0 클래스가 상당히 많은 것을 확인할 수 있음
- 기간 내에 갚은 대출의 비율이 많음
- 편향을 반영하기 위해 클래스에 weight 부여하기

## Missing value 처리

In [ ]:
# 각 column에 대해 결측값의 개수와 비율을 보여주는 함수
def missing_values_table(df):
        # 전체 데이터에 대한 null 값 개수
        mis_val = df.isnull().sum()
        
        # 비율 (percentage)
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        # 편하기 보기 위해 table 작성
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        
        # column 이름 재정의
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # 비율 기준으로 내림차순 정렬
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        
        # 결측치에 대한 정보가 담긴 data frame 리턴
        return mis_val_table_ren_columns

In [ ]:
# 위의 함수 적용해서 결측치에 관한 table 확인
missing_values = missing_values_table(app_train)
missing_values.head(20)

## Column 별로 type 확인

In [ ]:
# train data의 각 타입 별 column의 개수
app_train.dtypes.value_counts()

In [ ]:
# 범주형 변수의 개수 확인
app_train.select_dtypes('object').apply(pd.Series.nunique, axis = 0)

대부분의 범주형 변수가 소수의 unique한 classes를 가지고 있음
이런 변수들을 다루기 위해 Label Encoding을 사용해야 함

## 범주형 변수 Encoding

In [ ]:
# Label Encoding
le = LabelEncoder()
le_count = 0

for col in app_train:
    if app_train[col].dtype == 'object':
        # 카테고리가 2개 이하일 경우에 encoding 진행
        if len(list(app_train[col].unique())) <= 2:
            le.fit(app_train[col])
            # train과 test 모두에 대해 진행
            app_train[col] = le.transform(app_train[col])
            app_test[col] = le.transform(app_test[col])
            
            # 인코딩된 라벨 개수 counting
            le_count += 1
            
print('%d columns were label encoded.' % le_count)

In [ ]:
# 범주형 변수들에 대한 One-hot Encoding
app_train = pd.get_dummies(app_train)
app_test = pd.get_dummies(app_test)

print('Training Features shape: ', app_train.shape)
print('Testing Features shape: ', app_test.shape)

### Train data와 Test 데이터 열 맞춰주기

두 데이터에 동일한 열이 있어야 함.
위에서 진행한 one-hot encoding은 test data에 없는 변수도 인코딩 해서 train의 열이 더 많아졌음 

In [ ]:
train_labels = app_train['TARGET']

# Ineer Join으로 정렬
app_train, app_test = app_train.align(app_test, join = 'inner', axis = 1)

app_train['TARGET'] = train_labels

print('Training Features shape: ', app_train.shape)
print('Testing Features shape: ', app_test.shape)

## 이상치 (Anomalies) 확인

EDA를 할 때, anomaly를 주의해야 함



In [ ]:
# DAYS_BIRTH 확인하기
## 현재의 대출 신청과 비교하여 기록 --> 음수
## 연토별 통계를 보기 위해 -1을 곱하고 365 (일수)로 나누기

(app_train['DAYS_BIRTH'] / -365).describe()

In [ ]:
# DAYS_EMPLOYED 확인하기
app_train['DAYS_EMPLOYED'].describe()

In [ ]:
# DAYS_EMPLOYED 그래프로 그려서 확인
app_train['DAYS_EMPLOYED'].plot.hist(title = 'Days Employment Histogram');
plt.xlabel('Days Employment');

max 값이 350000일 (약 958년)인 것을 확인

In [ ]:
# DAYS_EMPLOYED anomal 값과 정상 값 defalut 비율 확인

anom = app_train[app_train['DAYS_EMPLOYED'] == 365243]
non_anom = app_train[app_train['DAYS_EMPLOYED'] != 365243]
print('The non-anomalies default on %0.2f%% of loans' % (100 * non_anom['TARGET'].mean()))
print('The anomalies default on %0.2f%% of loans' % (100 * anom['TARGET'].mean()))
print('There are %d anomalous days of employment' % len(anom))

anomalies가 더 낮은 default 비율을 가지고 있음
- nan 값으로 채우고 
- nan인지 아닌지를 boolean으로 표기하여 처리

In [ ]:
# Anomaly flag를 나타낼 column 생성
app_train['DAYS_EMPLOYED_ANOM'] = app_train["DAYS_EMPLOYED"] == 365243

# Anomaly 값을 nan으로 교체
app_train['DAYS_EMPLOYED'].replace({365243: np.nan}, inplace = True)

app_train['DAYS_EMPLOYED'].plot.hist(title = 'Days Employment Histogram');
plt.xlabel('Days Employment');

In [ ]:
app_test['DAYS_EMPLOYED_ANOM'] = app_test["DAYS_EMPLOYED"] == 365243
app_test["DAYS_EMPLOYED"].replace({365243: np.nan}, inplace = True)

print('There are %d anomalies in the test data out of %d entries' % (app_test["DAYS_EMPLOYED_ANOM"].sum(), len(app_test)))

## Correlations 확인하기

In [ ]:
# Target column의 corr 확인하고 sort
correlations = app_train.corr()['TARGET'].sort_values()

print('Most Positive Correlations:\n', correlations.tail(15))
print('\nMost Negative Correlations:\n', correlations.head(15))

DAYS_BIRTH : 대출 시점의 고객의 태어난 일수 (연령)  
상관관계는 양수지만, 실제 값은 음수로 적혀있음 --> 연령이 높을 수록 돈을 잘 갚는다

In [ ]:
# DAYS_BIRTH 만 뜯어서 살펴보기
app_train['DAYS_BIRTH'] = abs(app_train['DAYS_BIRTH'])
app_train['DAYS_BIRTH'].corr(app_train['TARGET'])

### 위의 결과(나이가 들 수록 잘 갚는다) 자세히 살펴보기

In [ ]:
# 그래프로 살펴보기 - 고객의 연령
plt.style.use('fivethirtyeight')

plt.hist(app_train['DAYS_BIRTH'] / 365, edgecolor = 'k', bins = 25)
plt.title('Age of Client'); plt.xlabel('Age (years)'); plt.ylabel('Count');

In [ ]:
#### KED PLOT

In [ ]:
# kde 플롯으로 살펴보기
plt.figure(figsize = (10, 8))

# 기간 내에 잘 갚음
sns.kdeplot(app_train.loc[app_train['TARGET'] == 0, 'DAYS_BIRTH'] / 365, label = 'target == 0')

# 기간 내에 못 갚음
sns.kdeplot(app_train.loc[app_train['TARGET'] == 1, 'DAYS_BIRTH'] / 365, label = 'target == 1')

plt.xlabel('Age (years)'); plt.ylabel('Density'); plt.title('Distribution of Ages');

위의 KDE plot에서 target이 1인 곡선은 나이가 어릴 수록 급격하게 기울고 있음  
corr의 수치가 0.07이지만, 변수에 영향을 끼치고 있다고 판단할 수 있음

In [ ]:
# 평균 연령대별로 기간 내 대출 상환 실패 비율 확인
age_data = app_train[['TARGET', 'DAYS_BIRTH']]
age_data['YEARS_BIRTH'] = age_data['DAYS_BIRTH'] / 365

# 5세 단위로 binding
age_data['YEARS_BINNED'] = pd.cut(age_data['YEARS_BIRTH'], bins = np.linspace(20, 70, num = 11))
age_data.head(10)

In [ ]:
# 연령대 별로 groupby 하고 평균 구하기
age_groups  = age_data.groupby('YEARS_BINNED').mean()
age_groups

In [ ]:
plt.figure(figsize = (8, 8))

# 연령대 별로 평균 target에 대한 막대그래프 그리기
plt.bar(age_groups.index.astype(str), 100 * age_groups['TARGET'])

plt.xticks(rotation = 75); plt.xlabel('Age Group (years)'); plt.ylabel('Failure to Repay (%)')
plt.title('Failure to Repay by Age Group');

**나이와 상관이 있음이 분명하게 확인**  
젊은 사람 : 대출금 상환하지 않을 가능성 높음

## 강한 상관관계를 가진 변수 살펴보기

#### EXT_SOURCE 변수

In [ ]:
# Target과 EXT_SOURCE 변수들 corr 확인
ext_data = app_train[['TARGET', 'EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'DAYS_BIRTH']]
ext_data_corrs = ext_data.corr()
ext_data_corrs

In [ ]:
plt.figure(figsize = (8, 6))

# 히트맵으로 시각화
sns.heatmap(ext_data_corrs, cmap = plt.cm.RdYlBu_r, vmin = -0.25, annot = True, vmax = 0.6)
plt.title('Correlation Heatmap');

- EXT_SOURCE 3개 모두 target과 음의 관계 : 증가할 수록 상환 가능성 높음
- DAYS_BIRTH와 EXT_SOURCE_1이 양의 관계 : 점수의 요인 중 하나가 연령일 수 있음

In [ ]:
# 각 변수가 target에 주는 영향 시각화
plt.figure(figsize = (10, 12))

# 3개 변수에 모두 진행
for i, source in enumerate(['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']):
    
    plt.subplot(3, 1, i + 1)
    
    sns.kdeplot(app_train.loc[app_train['TARGET'] == 0, source], label = 'target == 0')
    sns.kdeplot(app_train.loc[app_train['TARGET'] == 1, source], label = 'target == 1')
    
    plt.title('Distribution of %s by Target Value' % source)
    plt.xlabel('%s' % source); plt.ylabel('Density');
    
plt.tight_layout(h_pad = 2.5)
    

**EXT_SOURCE_3**의 차이가 가장 큼  
상환할 가능성과 관계가 있음을 알 수 있는 부분

# Feature Engineering
## FE 1 : polynominal features 생성

In [ ]:
# polynominal feature에 대한 새로운 data frame 생성
poly_features = app_train[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'DAYS_BIRTH', 'TARGET']]
poly_features_test = app_test[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'DAYS_BIRTH']]

# null 값 처리를 위해 Imputer import
## imputer : null 값을 쉽게 처리
from sklearn.impute import SimpleImputer

# null 값을 mean으로 대체한다고 지정하고 imputer 객체 생성
imputer = SimpleImputer(strategy = 'median')

poly_target = poly_features['TARGET']

poly_features = poly_features.drop(columns = ['TARGET'])

# null 값 impute
poly_features = imputer.fit_transform(poly_features)
poly_features_test = imputer.transform(poly_features_test)

from sklearn.preprocessing import PolynomialFeatures
                                  
# polynominal 객체 생성
poly_transformer = PolynomialFeatures(degree = 3)

## FE 2 : Transform the features

In [ ]:
# poly_feature 학습
poly_transformer.fit(poly_features)

# feature 변형
poly_features = poly_transformer.transform(poly_features)
poly_features_test = poly_transformer.transform(poly_features_test)
print('Polynomial Features shape: ', poly_features.shape)

In [ ]:
poly_transformer.get_feature_names(input_features = ['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'DAYS_BIRTH'])[:15]

## FE 3 : Target에 넣어서 상관관계 살펴보기

In [ ]:
# feature에 대한 데이터 프레임 생성 : polynomial feature를 통해 유용할 수 있는 새로운 feature 생성
poly_features = pd.DataFrame(poly_features, 
                             columns = poly_transformer.get_feature_names(['EXT_SOURCE_1', 'EXT_SOURCE_2', 
                                                                           'EXT_SOURCE_3', 'DAYS_BIRTH']))

# target에 넣기
poly_features['TARGET'] = poly_target

# 새로운 feature 들과 상관관계 살펴보기
poly_corrs = poly_features.corr()['TARGET'].sort_values()

print(poly_corrs.head(10))
print(poly_corrs.tail(10))

새로운 변수 중에 원래의 특징보다 더 큰 상관관계가 있는 것을 파악 가능

## FE 4 : Polynominal Feature와 TRAIN/TEST Merge

In [ ]:
# test feature를 data frame에 넣기
poly_features_test = pd.DataFrame(poly_features_test, 
                                  columns = poly_transformer.get_feature_names(['EXT_SOURCE_1', 'EXT_SOURCE_2', 
                                                                                'EXT_SOURCE_3', 'DAYS_BIRTH']))

# traing data frame에 polynomial feature merge
poly_features['SK_ID_CURR'] = app_train['SK_ID_CURR']
app_train_poly = app_train.merge(poly_features, on = 'SK_ID_CURR', how = 'left')

# test data와도 merge
poly_features_test['SK_ID_CURR'] = app_test['SK_ID_CURR']
app_test_poly = app_test.merge(poly_features_test, on = 'SK_ID_CURR', how = 'left')

# data frame Inner Join으로 정렬
app_train_poly, app_test_poly = app_train_poly.align(app_test_poly, join = 'inner', axis = 1)

print('Training data with polynomial features shape: ', app_train_poly.shape)
print('Testing data with polynomial features shape:  ', app_test_poly.shape)

## FE 5 : CREDIT_INCOME_PERCENT 생성

In [ ]:
# 모델 훈련에 사용할 새로운 변수들 생성 
app_train_domain = app_train.copy()
app_test_domain = app_test.copy()

app_train_domain['CREDIT_INCOME_PERCENT'] = app_train_domain['AMT_CREDIT'] / app_train_domain['AMT_INCOME_TOTAL']
app_test_domain['CREDIT_INCOME_PERCENT'] = app_test_domain['AMT_CREDIT'] / app_test_domain['AMT_INCOME_TOTAL']

## FE 6 : ANNUITY_INCOME_PERCENT 생성

In [ ]:
app_train_domain['ANNUITY_INCOME_PERCENT'] = app_train_domain['AMT_ANNUITY'] / app_train_domain['AMT_INCOME_TOTAL']
app_test_domain['ANNUITY_INCOME_PERCENT'] = app_test_domain['AMT_ANNUITY'] / app_test_domain['AMT_INCOME_TOTAL']

## FE 7 : CREDIT_TERM 생성

In [ ]:
app_train_domain['CREDIT_TERM'] = app_train_domain['AMT_ANNUITY'] / app_train_domain['AMT_CREDIT']
app_test_domain['CREDIT_TERM'] = app_test_domain['AMT_ANNUITY'] / app_test_domain['AMT_CREDIT']

## FE 8 : DAYS_EMPLOYED_PERCENT 생성

In [ ]:
app_train_domain['DAYS_EMPLOYED_PERCENT'] = app_train_domain['DAYS_EMPLOYED'] / app_train_domain['DAYS_BIRTH']
app_test_domain['DAYS_EMPLOYED_PERCENT'] = app_test_domain['DAYS_EMPLOYED'] / app_test_domain['DAYS_BIRTH']

#### 새로운 변수들 시각화

In [ ]:
plt.figure(figsize = (12, 20))

for i, feature in enumerate(['CREDIT_INCOME_PERCENT', 'ANNUITY_INCOME_PERCENT', 'CREDIT_TERM', 'DAYS_EMPLOYED_PERCENT']):
    

    plt.subplot(4, 1, i + 1)
    
    # 상환 함
    sns.kdeplot(app_train_domain.loc[app_train_domain['TARGET'] == 0, feature], label = 'target == 0')
    # 상환 안 함
    sns.kdeplot(app_train_domain.loc[app_train_domain['TARGET'] == 1, feature], label = 'target == 1')

    plt.title('Distribution of %s by Target Value' % feature)
    plt.xlabel('%s' % feature); plt.ylabel('Density');
    
plt.tight_layout(h_pad = 2.5)

유용하다고 보여지진 않음

# 모델링

In [ ]:
# ids 추출
train_ids = app_train['SK_ID_CURR']
test_ids = app_test['SK_ID_CURR']

labels = app_train['TARGET']

app_train = app_train.drop(columns = ['SK_ID_CURR', 'TARGET'])
app_test = app_test.drop(columns = ['SK_ID_CURR'])        

In [ ]:
# Extract feature names
feature_names = list(app_train.columns)

# Convert to np arrays
app_train = np.array(app_train)
app_test = np.array(app_test)

In [ ]:
from sklearn.model_selection import train_test_split
train_x, valid_x, train_y, valid_y = train_test_split(app_train, labels, test_size=0.3, random_state=2020)

In [ ]:
train_x.shape, valid_x.shape

In [ ]:
from lightgbm import LGBMClassifier
clf = LGBMClassifier(
        n_jobs=-1,
        n_estimators=1000,
        learning_rate=0.02,
        num_leaves=32,
        subsample=0.8,
        max_depth=12,
        silent=-1,
        verbose=-1
        )

clf.fit(train_x, train_y, eval_set=[(train_x, train_y), (valid_x, valid_y)], eval_metric= 'auc', verbose= 100, 
        early_stopping_rounds= 100)

In [ ]:
preds = clf.predict_proba(app_test, axis=1)[:, 1 ]

In [ ]:
submission = pd.DataFrame({'SK_ID_CURR': test_ids, 'TARGET': preds})

In [ ]:
submission.head()

In [ ]:
submission.to_csv('BigData_EDA_FE_SHUN.csv', index=False)